<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_D3_MongoDB_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('Tested',datetime.now(pytz.timezone('Asia/Calcutta')))

Tested 2023-04-11 14:54:40.461346+05:30


##References
https://stackoverflow.com/questions/45142629/cant-connect-to-mongo-db-via-spark <br>
https://docs.mongodb.com/spark-connector/current/python-api/

#Packages

In [2]:
!pip3 install pymongo
from pymongo import MongoClient
import pandas as pd
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.1/492.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 25.0 MB/s eta 0:00:00


#Install & Test LOCAL MongoDB

In [3]:
!apt install mongodb > /dev/null
!service mongodb start



 * Starting database mongodb
   ...done.


In [24]:
!mongod -version

db version v3.6.8
git version: 8e540c0b6db93ce994cc548f000900bdc740f80a
OpenSSL version: OpenSSL 1.1.1f  31 Mar 2020
allocator: tcmalloc
modules: none
build environment:
    distarch: x86_64
    target_arch: x86_64


In [4]:
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local']

In [5]:
db = client.local
#db = client.sparkDB
#db = client.people
db.list_collection_names()

['startup_log']

In [6]:
table = db.startup_log
#table = db.coll01
#table = db.contacts
#table.count_documents({}) #gives the number of documents in the table
table.find_one()

{'_id': '5d8fb68efc15-1681205196575',
 'hostname': '5d8fb68efc15',
 'startTime': datetime.datetime(2023, 4, 11, 9, 26, 36),
 'startTimeLocal': 'Tue Apr 11 09:26:36.575',
 'cmdLine': {'config': '/etc/mongodb.conf',
  'net': {'bindIp': '127.0.0.1'},
  'storage': {'dbPath': '/var/lib/mongodb', 'journal': {'enabled': True}},
  'systemLog': {'destination': 'file',
   'logAppend': True,
   'path': '/var/log/mongodb/mongodb.log'}},
 'pid': 1940,
 'buildinfo': {'version': '3.6.8',
  'gitVersion': '8e540c0b6db93ce994cc548f000900bdc740f80a',
  'modules': [],
  'allocator': 'tcmalloc',
  'javascriptEngine': 'mozjs',
  'sysInfo': 'deprecated',
  'versionArray': [3, 6, 8, 0],
  'openssl': {'running': 'OpenSSL 1.1.1f  31 Mar 2020',
   'compiled': 'OpenSSL 1.1.1f  31 Mar 2020'},
  'buildEnvironment': {'distmod': '',
   'distarch': 'x86_64',
   'cc': 'cc: cc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0',
   'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unkno

#Connect to and Test Remote MongoDB

#Install Spark

## Deprecated - dont use these commands

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#

#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark              --- no more required,  replaced by pyspark
!pip install -q pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 55.3 MB/s 


## Simple PIP3 install

In [7]:
!pip3 install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.2 MB/s eta 0:00:00


#Spark2Mongo

In [10]:
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/sparkDB.coll01") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/sparkDB.coll01") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
    .getOrCreate()

In [ ]:
#sc.stop()

In [11]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

##Write to MongoDB

In [12]:
people = spark.createDataFrame([("Bilbo Baggins",  50), ("Gandalf", 1000), ("Thorin", 195), ("Balin", 178), ("Kili", 77),
   ("Dwalin", 169), ("Oin", 167), ("Gloin", 158), ("Fili", 82), ("Bombur", None)], ["name", "age"])

In [13]:
people.show()

+-------------+----+
|         name| age|
+-------------+----+
|Bilbo Baggins|  50|
|      Gandalf|1000|
|       Thorin| 195|
|        Balin| 178|
|         Kili|  77|
|       Dwalin| 169|
|          Oin| 167|
|        Gloin| 158|
|         Fili|  82|
|       Bombur|null|
+-------------+----+



In [14]:
# this will send the data to the default database / collection specified during startup
people.write.format("mongo").mode("append").save()

In [15]:
# this will send the data to a new database / collection specified in this command
people.write.format("mongo").mode("append").option("database","people").option("collection", "contacts").save()

In [16]:
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local', 'people', 'sparkDB']

##Check with MongoDB client

In [17]:
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local', 'people', 'sparkDB']

In [18]:
db = client.people
list(db.list_collections())

[{'name': 'contacts',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'\xf7\xfc\xb5\xa8\x12\xa9DD\x9f\x19\x86\x08#\xe6b\r', 4)},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'people.contacts'}}]

In [19]:
db2 = client.sparkDB
list(db2.list_collections())

[{'name': 'coll01',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'\xd1\x80\xcba:\xf2D\x0c\xa3\xa0_%\x94DQ\xa0', 4)},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'sparkDB.coll01'}}]

In [20]:
kollection = db.contacts
c = kollection.find({},{"_id":0})
for crnt in c:
    print(crnt)

{'name': 'Bilbo Baggins', 'age': 50}
{'name': 'Gandalf', 'age': 1000}
{'name': 'Thorin', 'age': 195}
{'name': 'Balin', 'age': 178}
{'name': 'Kili', 'age': 77}
{'name': 'Dwalin', 'age': 169}
{'name': 'Oin', 'age': 167}
{'name': 'Gloin', 'age': 158}
{'name': 'Fili', 'age': 82}
{'name': 'Bombur'}


In [21]:
pandasDF = pd.DataFrame(kollection.find({},{"_id":0}))
pandasDF

,name,age
0,Bilbo Baggins,50.0
1,Gandalf,1000.0
2,Thorin,195.0
3,Balin,178.0
4,Kili,77.0
5,Dwalin,169.0
6,Oin,167.0
7,Gloin,158.0
8,Fili,82.0
9,Bombur,NaN


##Read from MongoDB

In [22]:
sparkDF = spark.read.format("mongo").option("uri","mongodb://127.0.0.1/people.contacts").load()
sparkDF.show()

+--------------------+----+-------------+
|                 _id| age|         name|
+--------------------+----+-------------+
|{643529b487e5a700...|  50|Bilbo Baggins|
|{643529b487e5a700...|1000|      Gandalf|
|{643529b487e5a700...| 195|       Thorin|
|{643529b487e5a700...| 178|        Balin|
|{643529b487e5a700...|  77|         Kili|
|{643529b487e5a700...| 169|       Dwalin|
|{643529b487e5a700...| 167|          Oin|
|{643529b487e5a700...| 158|        Gloin|
|{643529b487e5a700...|  82|         Fili|
|{643529b487e5a700...|null|       Bombur|
+--------------------+----+-------------+



In [23]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

signed off at   2023-04-11 15:05:31.367749+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)